In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Attention, Bidirectional, Dropout
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Загрузка данных
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=10000)

In [3]:
# Предобработка данных
max_len = 200
x_train = pad_sequences(x_train, maxlen=max_len, truncating='post')
x_test = pad_sequences(x_test, maxlen=max_len, truncating='post')

In [4]:
# Модель RNN с attention
embedding_dim = 100
hidden_dim = 256
output_dim = 1
dropout_rate = 0.5

In [5]:
inputs = Input(shape=(max_len,))
embedding = Embedding(input_dim=10000, output_dim=embedding_dim, input_length=max_len)(inputs)
lstm = Bidirectional(LSTM(hidden_dim, return_sequences=True))(embedding)
attention = Attention()([lstm, lstm])
context = tf.reduce_sum(attention * lstm, axis=1)
dropout = Dropout(dropout_rate)(context)
output = Dense(output_dim, activation='sigmoid')(dropout)
model = Model(inputs=inputs, outputs=output)

In [6]:
# Компиляция модели
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
# Добавленные строки для сохранения весов внимания
attention_weights = []

In [8]:
# Обучение модели
for epoch in range(3):
    history = model.fit(x_train, y_train, epochs=1, batch_size=64, validation_data=(x_test, y_test))

    # Получение весов внимания
    attention_layer = model.layers[2]  # Получаем attention слой
    weights = attention_layer.get_weights()[0]
    mean_attention_weight = np.mean(weights)
    attention_weights.append(mean_attention_weight)



391/391 [==============================] - 305s 779ms/step - loss: 0.1898 - accuracy: 0.9241 - val_loss: 0.4803 - val_accuracy: 0.8113


In [9]:
# Оценка точности модели
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test Loss: {loss:.3f}, Test Accuracy: {accuracy * 100:.2f}%')

782/782 [==============================] - 97s 123ms/step - loss: 0.4803 - accuracy: 0.8113
Test Loss: 0.480, Test Accuracy: 81.13%


In [11]:
attention_weights

[-0.00066940946, -0.00023228649, -0.000120513665]

In [12]:
accuracy

0.8112800121307373